In [15]:
#import tensorflow as tf
import os
import pandas as pd
import json

In [26]:
balanced_train_segments_path = "c:/Users/sbrxb/waveled/data/raw/balanced_train_segments.csv"
ontology_path = "c:/Users/sbrxb/waveled/data/raw/ontology.json"


In [27]:
# Ruta al directorio que contiene los archivos .tfrecord
base_dir = "c:/Users/sbrxb/waveled/data/raw/bal_train"  


files = os.listdir(base_dir)
print("Archivos en el directorio:", files)
print(len(files))

Archivos en el directorio: ['--.tfrecord', '-0.tfrecord', '-1.tfrecord', '-2.tfrecord', '-3.tfrecord', '-4.tfrecord', '-5.tfrecord', '-6.tfrecord', '-7.tfrecord', '-8.tfrecord', '-9.tfrecord', '-a.tfrecord', '-A_1.tfrecord', '-b.tfrecord', '-B_1.tfrecord', '-C.tfrecord', '-c_1.tfrecord', '-D.tfrecord', '-d_1.tfrecord', '-e.tfrecord', '-E_1.tfrecord', '-f.tfrecord', '-F_1.tfrecord', '-g.tfrecord', '-G_1.tfrecord', '-H.tfrecord', '-h_1.tfrecord', '-I.tfrecord', '-i_1.tfrecord', '-J.tfrecord', '-j_1.tfrecord', '-k.tfrecord', '-K_1.tfrecord', '-L.tfrecord', '-l_1.tfrecord', '-m.tfrecord', '-M_1.tfrecord', '-n.tfrecord', '-N_1.tfrecord', '-O.tfrecord', '-o_1.tfrecord', '-P.tfrecord', '-p_1.tfrecord', '-q.tfrecord', '-Q_1.tfrecord', '-r.tfrecord', '-R_1.tfrecord', '-S.tfrecord', '-s_1.tfrecord', '-T.tfrecord', '-t_1.tfrecord', '-U.tfrecord', '-u_1.tfrecord', '-V.tfrecord', '-v_1.tfrecord', '-W.tfrecord', '-w_1.tfrecord', '-x.tfrecord', '-X_1.tfrecord', '-Y.tfrecord', '-y_1.tfrecord', '-z.tfr

In [28]:
with open(balanced_train_segments_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Leer el archivo línea por línea y reconstruir las filas correctamente
fixed_rows = []
for line in lines[3:]:  # Omitir los primeros 3 encabezados
    parts = line.strip().split(",")  # Separar por comas
    if len(parts) >= 4:  # Asegurar que haya al menos 4 partes
        fixed_rows.append([parts[0], parts[1], parts[2], ",".join(parts[3:])])  # Unir la última parte como string

# Crear un DataFrame manualmente
df_segments = pd.DataFrame(fixed_rows, columns=["YTID", "start_seconds", "end_seconds", "positive_labels"])

# Mostrar las primeras filas
df_segments.head()

,YTID,start_seconds,end_seconds,positive_labels
0,--PJHxphWEs,30.000,40.000,"""/m/09x0r,/t/dd00088"""
1,--ZhevVpy1s,50.000,60.000,"""/m/012xff"""
2,--aE2O5G5WE,0.000,10.000,"""/m/03fwl,/m/04rlf,/m/09x0r"""
3,--aO5cdqSAg,30.000,40.000,"""/t/dd00003,/t/dd00005"""
4,--aaILOrkII,200.000,210.000,"""/m/032s66,/m/073cg4"""


Estructura de los datos:

YTID: ID del video en YouTube.

start_seconds: Segundo de inicio del segmento de audio.

end_seconds: Segundo de fin del segmento de audio.

positive_labels: Etiquetas asociadas al audio.

## ontology JSON, contiene información de cada id

In [21]:
# Cargar el JSON ontology
with open(ontology_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Crear el DataFrame
df_ontology = pd.DataFrame(data)

# Filtrar por la palabra 'music' en la columna 'name'
df_ontology = df_ontology[df_ontology["name"].str.contains("music", case=False, na=False)]

df_ontology.head(5)


,id,name,description,citation_uri,positive_examples,child_ids,restrictions
156,/m/04rlf,Music,Music is an art form and cultural activity who...,http://en.wikipedia.org/wiki/Music,"[youtu.be/E83fRA1wxKU?start=420&end=430, youtu...","[/m/04szw, /m/0kpv1t, /t/dd00027, /t/dd00028, ...",[]
157,/m/04szw,Musical instrument,Sounds specifically associated with instrument...,http://en.wikipedia.org/wiki/Musical_instrument,"[youtu.be/7umb9yJpkKU?start=70&end=80, youtu.b...","[/m/0fx80y, /m/05148p4, /m/0l14md, /m/05pd6, /...",[]
171,/m/05148p4,Keyboard (musical),Sounds of a musical instrument played using a ...,http://en.wikipedia.org/wiki/Keyboard_instrument,"[youtu.be/11bWvQaQhrM?start=440&end=450, youtu...","[/m/05r5c, /m/013y1f, /m/0l14qv, /m/03q5t]",[]
244,/m/05229,Musical ensemble,The sound of a group of people who perform ins...,http://en.wikipedia.org/wiki/Musical_ensemble,[],[],[blacklist]
247,/m/0kpv1t,Music genre,Portmanteau class holding categories that repr...,,[],"[/m/064t9, /m/0glt670, /m/06by7, /m/06j6l, /m/...",[abstract]


In [30]:
ids_music = set(df_ontology["id"])

df_segments_filtered = df_segments[df_segments["positive_labels"].apply(lambda x: any(id_ in x.split(",") for id_ in ids_music))]
df_segments_filtered


,YTID,start_seconds,end_seconds,positive_labels
2,--aE2O5G5WE,0.000,10.000,"""/m/03fwl,/m/04rlf,/m/09x0r"""
11,-0SdAVK79lg,30.000,40.000,"""/m/0155w,/m/01lyv,/m/0342h,/m/042v_gx,/m/04r..."
13,-0mG4W5Hlq8,270.000,280.000,"""/m/04rlf,/m/05fw6t,/m/07r4k75,/m/09x0r,/m/0y..."
17,-1TLtjPtnms,10.000,20.000,"""/m/03lty,/m/04rlf,/m/07szfh9"""
43,-5xOcMJpTUk,70.000,80.000,"""/m/018vs,/m/0342h,/m/042v_gx,/m/04rlf,/m/04s..."
...,...,...,...,...
22095,zo1D2kSxnxs,60.000,70.000,"""/m/04rlf,/m/07lnk,/m/07s72n,/m/0cfdd"""
22105,zqga01RTsB4,20.000,30.000,"""/m/0283d,/m/03mb9,/m/04rlf,/m/07gxw,/m/07lnk..."
22121,ztTVZolaOAg,80.000,90.000,"""/m/0342h,/m/04rlf,/m/04szw,/m/05r5wn,/m/0fx80y"""
22134,zw-cQWbrGBE,20.000,30.000,"""/g/122z_qxw,/m/04rlf,/m/07qsvvw"""
